In [1]:
"""
Description: Script to optimize Duca's currency mix.
Author: Jeroen van Dijk & Victor de Graaff
Date: 04-11-2020
Maintainer: Jeroen van Dijk & Victor de Graaff
Email: jeroen.vandijk@d-data.nl & victor.degraaff@d-data.nl
Status: Dev
"""

"\nDescription: Script to optimize Duca's currency mix.\nAuthor: Jeroen van Dijk & Victor de Graaff\nDate: 04-11-2020\nMaintainer: Jeroen van Dijk & Victor de Graaff\nEmail: jeroen.vandijk@d-data.nl & victor.degraaff@d-data.nl\nStatus: Dev\n"

In [2]:
# Load common imports
%run ./CommonImports.ipynb

In [3]:
# Load common functions and currencies lists:
# - all_currencies
# - obsolete_currencies
# - p13_currencies
# - f_currencies
# - ff_currencies
# - currencies_per_continent
%run ./Utilities.ipynb

In [4]:
# Load loss functions
# - calculate_loss_function_around_one(weights)
# - calculate_loss_function_vs_t_minus_one(weights)
# - calculate_loss_function_vs_t_minus_one_with_balancing(weights)
# - calculate_loss_function_vs_t_minus_one_for_period(weights, max_date)
%run ./LossFunctions.ipynb

In [5]:
from scipy.stats import pearsonr

currency_pairs = [(c1, c2) for c1 in all_currencies for c2 in all_currencies if c1 > c2]

highly_correlated_currencies = set()
highly_unstable_currencies = set()

for base_currency in all_currencies:
    print(f"Evaluating correlations using base currency {base_currency}")
    df = create_original_df(base_currency)
    
    median = np.median(np.array([pearsonr(df[c1], df[c2])[0] 
                                 for c1, c2 in currency_pairs 
                                 if c1 != base_currency and c2 != base_currency]))
    
    if median > .9:
        highly_unstable_currencies.add(base_currency)
        continue

    for c1, c2 in currency_pairs:
        if base_currency == c1 or base_currency == c2:
            continue
            
        if c1 in highly_correlated_currencies or c2 in highly_correlated_currencies:
            continue
            
        corr = pearsonr(df[c1], df[c2])[0]

        if abs(corr) > .98:
            if trading_volumes[c1] >= trading_volumes[c2] and \
               trading_volumes[base_currency] >= trading_volumes[c2]:
                print(f"According to {base_currency} {c1} and {c2} are highly corrolated:", corr)
                highly_correlated_currencies.add(c2)
            elif trading_volumes[c2] >= trading_volumes[c1] and \
                 trading_volumes[base_currency] >= trading_volumes[c1]:
                print(f"According to {base_currency} {c2} and {c1} are highly corrolated:", corr)
                highly_correlated_currencies.add(c1)

    print(highly_unstable_currencies, highly_correlated_currencies)
    
highly_unstable_currencies, highly_correlated_currencies

Evaluating correlations using base currency CZK
According to CZK EUR and HRK are highly corrolated: 0.9882185402623102
set() {'HRK'}
Evaluating correlations using base currency CHF
According to CHF EUR and DKK are highly corrolated: 0.9999514621476165
According to CHF USD and HKD are highly corrolated: 0.9997956628000464
According to CHF EUR and RON are highly corrolated: 0.9862360542626356
According to CHF NOK and RUB are highly corrolated: 0.9803307030132263
set() {'DKK', 'HKD', 'RUB', 'RON', 'HRK'}
Evaluating correlations using base currency TRY
Evaluating correlations using base currency MYR
{'TRY'} {'DKK', 'HKD', 'RUB', 'RON', 'HRK'}
Evaluating correlations using base currency KRW
{'TRY'} {'DKK', 'HKD', 'RUB', 'RON', 'HRK'}
Evaluating correlations using base currency THB
{'TRY'} {'DKK', 'HKD', 'RUB', 'RON', 'HRK'}
Evaluating correlations using base currency EUR
{'TRY'} {'DKK', 'HKD', 'RUB', 'RON', 'HRK'}
Evaluating correlations using base currency PHP
{'TRY'} {'DKK', 'HKD', 'RUB',

({'TRY', 'ZAR'}, {'DKK', 'HKD', 'HRK', 'MYR', 'PHP', 'RON', 'RUB', 'THB'})

In [6]:
# Create dataframe with pre-defined base currency
df = create_original_df("USD")

In [7]:
# Determine splits for dev- (train and test) and validation set
train_start = date(2005, 4, 1)
test_start = date(2018, 1, 1)
validation_start = date(2018, 1, 1)
validation_end = date(2020, 10, 1)

# Create dev (train and test) and validation set
train, test, dev, val = split_data(df, train_start, test_start, validation_start, validation_end)

In [95]:
# Create exchange table for each currency
exchange_table = dict()

for base_currency in all_currencies:
    exchange_table[base_currency] = 1 / train[all_currencies].divide(train[base_currency], axis=0)

# Preview USD exchange table
exchange_table["USD"]

,CZK,CHF,TRY,MYR,KRW,THB,EUR,PHP,DKK,HKD,NOK,SGD,USD,RON,GBP,AUD,JPY,CNY,HUF,IDR,RUB,NZD,HRK,SEK,PLN,CAD,ZAR
date,,,,,,,,,,,,,,,,,,,,,,,,,,,
2005-04-01,0.04315,0.83461,0.73963,0.26316,0.00099,0.02554,1.29590,0.01827,0.17394,0.12823,0.15813,0.60418,1.00000,0.35967,1.88728,0.77123,0.00932,0.12082,0.00524,0.00011,0.03588,0.71051,0.17460,0.14134,0.31703,0.82353,0.16175
2005-04-02,0.04307,0.83284,0.73758,0.26316,0.00099,0.02546,1.29337,0.01826,0.17360,0.12822,0.15780,0.60293,1.00000,0.35897,1.88327,0.77012,0.00930,0.12082,0.00523,0.00011,0.03587,0.70945,0.17424,0.14100,0.31580,0.82270,0.16153
2005-04-03,0.04299,0.83106,0.73553,0.26317,0.00099,0.02537,1.29083,0.01825,0.17326,0.12822,0.15748,0.60168,1.00000,0.35827,1.87926,0.76901,0.00928,0.12082,0.00522,0.00011,0.03586,0.70838,0.17387,0.14065,0.31458,0.82187,0.16132
2005-04-04,0.04291,0.82929,0.73349,0.26317,0.00099,0.02529,1.28830,0.01824,0.17292,0.12821,0.15715,0.60044,1.00000,0.35756,1.87525,0.76790,0.00926,0.12082,0.00521,0.00011,0.03585,0.70731,0.17351,0.14031,0.31336,0.82104,0.16110
2005-04-05,0.04271,0.82427,0.73137,0.26316,0.00098,0.02522,1.28100,0.01824,0.17195,0.12821,0.15691,0.60059,1.00000,0.35554,1.87253,0.76441,0.00921,0.12082,0.00518,0.00011,0.03579,0.70431,0.17266,0.13960,0.31019,0.81681,0.16014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-27,0.04603,1.01045,0.26157,0.24535,0.00093,0.03048,1.18950,0.02002,0.15977,0.12797,0.12063,0.74638,1.00000,0.25581,1.34266,0.77664,0.00883,0.15255,0.00382,0.00007,0.01734,0.70698,0.15776,0.12048,0.28400,0.79152,0.08031
2017-12-28,0.04654,1.01965,0.26252,0.24591,0.00093,0.03066,1.19340,0.02004,0.16028,0.12795,0.12095,0.74737,1.00000,0.25653,1.34440,0.77847,0.00886,0.15300,0.00385,0.00007,0.01735,0.70800,0.15888,0.12122,0.28545,0.79301,0.08100
2017-12-29,0.04697,1.02487,0.26379,0.24709,0.00094,0.03066,1.19930,0.02006,0.16109,0.12797,0.12188,0.74844,1.00000,0.25744,1.35174,0.78151,0.00888,0.15367,0.00386,0.00007,0.01728,0.71175,0.16120,0.12183,0.28712,0.79746,0.08100


### Set the reference_exchange_rate to the median w.r.t. the USD

In [9]:
candidate_currencies = [c for c in all_currencies if c not in highly_correlated_currencies and
                                                     c not in highly_unstable_currencies]
more_stable_currencies = [c for c in all_currencies if c not in highly_unstable_currencies]
candidate_currencies.sort()

reference_exhange_rate = train[more_stable_currencies].median()
reference_exhange_rate

CZK      20.37242
CHF       0.99263
MYR       3.44649
KRW   1,110.86580
THB      32.91149
EUR       0.76994
PHP      45.94996
DKK       5.73840
HKD       7.76032
NOK       6.16722
SGD       1.37649
USD       1.00000
RON       3.24164
GBP       0.62603
AUD       1.19073
JPY     104.37078
CNY       6.68739
HUF     219.10857
IDR   9,682.09762
RUB      30.91923
NZD       1.37283
HRK       5.71817
SEK       7.13566
PLN       3.16782
CAD       1.10118
dtype: float64

In [10]:
def run_optimization(selected_currencies, 
                     max_iter=100, 
                     loss_function=calculate_loss_function_around_one,
                     reference_currencies=all_currencies):
    starting_point = np.ones(len(selected_currencies))/len(selected_currencies)
    bounds = [(0, 1) for _ in range(len(starting_point))]
    
    result = minimize(fun=loss_function, 
                      x0=starting_point, 
                      args=(selected_currencies, reference_currencies),
                      bounds=bounds,
                      options={"disp": True, 
                               "maxiter": max_iter})
    
    output = pd.Series(result.x/result.x.sum(), index=reference_exhange_rate.loc[selected_currencies].index)
    output = output.sort_values(ascending=False)
        
    return output

In [11]:
def try_currencies(currencies_to_try, currencies, reference_currencies):
    for currency in currencies_to_try:
        if currency in currencies:
            return None, 1

    new_mix = run_optimization(selected_currencies=currencies + currencies_to_try, 
                               max_iter=100, 
                               loss_function=calculate_loss_function_around_one,
                               reference_currencies=reference_currencies)

    new_score = calculate_loss_function_around_one(new_mix, currencies + currencies_to_try, reference_currencies)

    return new_mix, new_score

In [75]:
result = pd.DataFrame()

with open("logs/output-loss-function-on-more-stable-currencies-0_0005-0_999.log", "a") as output:
    for currency in candidate_currencies:
        selected_currencies_so_far = [currency]
        best_score = 1
        last_score = 1

        while best_score == 1 or best_candidate is not None:
            output.write(f"Attempting to improve set, starting from: {selected_currencies_so_far}\n")
            print(f"Attempting to improve set, starting from: {selected_currencies_so_far}")
            best_candidate = None

            for candidate_currency in candidate_currencies:
                new_mix, new_score = try_currencies([candidate_currency], 
                                                    selected_currencies_so_far, 
                                                    more_stable_currencies)

                if (new_score < .999 * last_score) and (new_score < best_score):
                    best_score = new_score
                    best_candidate = candidate_currency
                    best_mix = new_mix

            last_score = best_score

            if best_candidate is not None:
                selected_currencies_so_far += [best_candidate]
                
            if len(selected_currencies_so_far) == 7:
                # Let's make sure this process ever finishes
                break

        output.write(f"Best set: {selected_currencies_so_far} ({best_score})\nWith mix:\n{best_mix}")
        print(f"Best set: {selected_currencies_so_far} ({best_score})\nWith mix:\n{best_mix}")
        
        result = result.append({"mix": best_mix, "score": best_score}, ignore_index=True)
        
result.sort_values("score")["mix"].iloc[0]
        

Attempting to improve set, starting from: ['AUD']
Attempting to improve set, starting from: ['AUD', 'USD']
Attempting to improve set, starting from: ['AUD', 'USD', 'EUR']
Attempting to improve set, starting from: ['AUD', 'USD', 'EUR', 'CAD']
Attempting to improve set, starting from: ['AUD', 'USD', 'EUR', 'CAD', 'JPY']
Attempting to improve set, starting from: ['AUD', 'USD', 'EUR', 'CAD', 'JPY', 'SEK']
Best set: ['AUD', 'USD', 'EUR', 'CAD', 'JPY', 'SEK', 'CNY'] (0.005895438711103024)
With mix:
USD   0.49547
EUR   0.19016
SEK   0.10447
CAD   0.09213
JPY   0.06316
CNY   0.04325
AUD   0.01136
dtype: float64
Attempting to improve set, starting from: ['CAD']
Attempting to improve set, starting from: ['CAD', 'USD']
Attempting to improve set, starting from: ['CAD', 'USD', 'EUR']
Attempting to improve set, starting from: ['CAD', 'USD', 'EUR', 'JPY']
Attempting to improve set, starting from: ['CAD', 'USD', 'EUR', 'JPY', 'NZD']
Attempting to improve set, starting from: ['CAD', 'USD', 'EUR', 'JPY'

USD   0.36044
EUR   0.18250
GBP   0.11510
SGD   0.11048
JPY   0.08594
CNY   0.08536
IDR   0.06018
dtype: float64

### Comparison and validation

#### Development set

In [111]:
duca_mix = result.sort_values("score")["mix"].iloc[0]

duca_mix.index.name = "currency"
duca_mix.name = "weight"
duca_mix = duca_mix.map(lambda x: round(x, 4))
duca_mix.to_csv("duca_mix.csv", sep=";")

duca_mix

currency
USD   0.36040
EUR   0.18250
GBP   0.11510
SGD   0.11050
JPY   0.08590
CNY   0.08540
IDR   0.06020
Name: weight, dtype: float64

In [112]:
calculate_loss_function_around_one(duca_mix,
                                   selected_currencies=duca_mix_new.index,
                                   reference_currencies=more_stable_currencies)

0.005842938635213402

In [113]:
libra_mix = get_libra_mix(all_currencies)

calculate_loss_function_around_one(libra_mix,
                                   selected_currencies=libra_mix.index,
                                   reference_currencies=more_stable_currencies)

0.005966469598100568

In [114]:
sdr_mix = get_sdr_mix(all_currencies)

calculate_loss_function_around_one(sdr_mix,
                                   selected_currencies=sdr_mix.index,
                                   reference_currencies=more_stable_currencies)

0.005937814504026259

#### Validation set

In [115]:
# Create exchange table for each currency
exchange_table = dict()

for base_currency in all_currencies:
    exchange_table[base_currency] = 1/val[all_currencies].divide(val[base_currency], axis=0)

# Preview USD exchange table
exchange_table["USD"]

,CZK,CHF,TRY,MYR,KRW,THB,EUR,PHP,DKK,HKD,NOK,SGD,USD,RON,GBP,AUD,JPY,CNY,HUF,IDR,RUB,NZD,HRK,SEK,PLN,CAD,ZAR
date,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-01-01,0.04724,1.02843,0.26552,0.24836,0.00094,0.03080,1.20470,0.02006,0.16184,0.12797,0.12304,0.75156,1.00000,0.25885,1.35519,0.78246,0.00891,0.15393,0.00390,0.00007,0.01741,0.71163,0.16153,0.12253,0.28912,0.79751,0.08098
2018-01-02,0.04732,1.02961,0.26610,0.24879,0.00094,0.03084,1.20650,0.02006,0.16208,0.12797,0.12343,0.75260,1.00000,0.25932,1.35633,0.78278,0.00891,0.15401,0.00391,0.00007,0.01746,0.71159,0.16164,0.12276,0.28979,0.79753,0.08097
2018-01-03,0.04707,1.02445,0.26539,0.24907,0.00094,0.03074,1.20230,0.02004,0.16151,0.12792,0.12339,0.75200,1.00000,0.25937,1.35639,0.78382,0.00891,0.15381,0.00389,0.00007,0.01740,0.70966,0.16158,0.12237,0.28865,0.79903,0.08078
2018-01-04,0.04729,1.02567,0.26594,0.24925,0.00094,0.03104,1.20650,0.02009,0.16206,0.12791,0.12355,0.75204,1.00000,0.26064,1.35405,0.78354,0.00888,0.15393,0.00391,0.00007,0.01750,0.71151,0.16226,0.12284,0.29034,0.79827,0.08126
2018-01-05,0.04706,1.02450,0.26691,0.25000,0.00094,0.03107,1.20450,0.02005,0.16177,0.12788,0.12364,0.75314,1.00000,0.25986,1.35515,0.78413,0.00883,0.15412,0.00390,0.00007,0.01751,0.71539,0.16200,0.12251,0.28986,0.79938,0.08090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-26,0.04295,1.07793,0.13013,0.23980,0.00085,0.03161,1.16460,0.02062,0.15640,0.12903,0.10477,0.72678,1.00000,0.23889,1.27887,0.70427,0.00948,0.14654,0.00320,0.00007,0.01282,0.65546,0.15427,0.10972,0.25574,0.74750,0.05853
2020-09-27,0.04298,1.07845,0.12912,0.23964,0.00085,0.03159,1.16580,0.02061,0.15657,0.12903,0.10515,0.72744,1.00000,0.23916,1.28411,0.70549,0.00948,0.14666,0.00320,0.00007,0.01276,0.65571,0.15442,0.10998,0.25611,0.74731,0.05860
2020-09-28,0.04301,1.07896,0.12814,0.23949,0.00086,0.03157,1.16700,0.02061,0.15673,0.12903,0.10553,0.72810,1.00000,0.23942,1.28939,0.70672,0.00949,0.14677,0.00320,0.00007,0.01271,0.65595,0.15457,0.11024,0.25647,0.74712,0.05867


In [116]:
calculate_loss_function_around_one(duca_mix,
                                   selected_currencies=duca_mix_new.index,
                                   reference_currencies=more_stable_currencies)

0.009631346384278483

In [117]:
libra_mix = get_libra_mix(all_currencies)

calculate_loss_function_around_one(libra_mix,
                                   selected_currencies=libra_mix.index,
                                   reference_currencies=more_stable_currencies)

0.009951065009372243

In [118]:
sdr_mix = get_sdr_mix(all_currencies)

calculate_loss_function_around_one(sdr_mix,
                                   selected_currencies=sdr_mix.index,
                                   reference_currencies=more_stable_currencies)

0.009692705580961336